In [1]:
# import dependencies
from resources.config import strUsername, strPassword, strToken
from splinter import Browser
import lib.MakeUserTable
import pandas as pd
from simple_salesforce import Salesforce

In [2]:
# connect to Salesforce API
sf = Salesforce(username=strUsername, password=strPassword, security_token=strToken)

In [3]:
# get active users
lstActiveUser = sf.query("SELECT Id FROM User WHERE Profile.UserLicense.Name = 'Salesforce' AND IsActive = TRUE")

In [4]:
# set up splinter browser
dictPath = {'executable_path': 'chromedriver.exe'}
brsr = Browser('chrome', **dictPath, headless=False)

In [5]:
# log in to Salesforce
strURLLogin = 'https://login.salesforce.com/'
brsr.visit(strURLLogin)
brsr.fill('username', strUsername)
brsr.fill('pw', strPassword)
btnLogin = brsr.find_by_name('Login')
btnLogin.click()

In [6]:
# set up master dataframe
dfUsers = pd.DataFrame(columns=['Record Type', 'Record Count'])

In [7]:
# loop through all Active Users
for rec in lstActiveUser['records']:
    # visit User Storage webpage
    strId = rec['Id']
    strURLUser = 'https://na1.salesforce.com/setup/user/userstorageusage.jsp?id=' + strId
    brsr.visit(strURLUser)
    html = brsr.html
    dfUser = lib.MakeUserTable.MakeUserTable(html, strId)
    dfUsers = pd.concat([dfUser, dfUsers])

In [8]:
dfUsers.to_excel('export.xlsx')

In [9]:
# quit
brsr.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x000001BF18C307C8>>